In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

customers = pd.read_csv('/content/Customers.csv')
transactions = pd.read_csv('//content/Transactions.csv')

In [3]:
transaction = transactions.groupby('CustomerID').agg(
    TotalSpent=('TotalValue', 'sum'),
    TotalQuantity=('Quantity', 'sum'),
    Transactions=('TransactionID', 'count')
).reset_index()

customerp = pd.merge(customers, transaction, on='CustomerID', how='inner')

In [4]:
numericfeatures = ['TotalSpent', 'TotalQuantity', 'Transactions']
scaler = StandardScaler()
customerp[numericfeatures] = scaler.fit_transform(customerp[numericfeatures])

In [5]:
def recommendc(inp, profiles, topn=3):
    features = profiles[numericfeatures]
    similaritym = cosine_similarity(features)
    customerindex = profiles.index[profiles['CustomerID'] == inp][0]
    similarityscores = list(enumerate(similaritym[customerindex]))
    similarc = sorted(similarityscores, key=lambda x: x[1], reverse=True)

    topsimilar = [
        (profiles.iloc[idx]['CustomerID'], score)
        for idx, score in similarc if idx != customerindex
    ][:topn]

    return pd.DataFrame(topsimilar, columns=['CustomerID', 'SimilarityScore'])



In [6]:
toplookalikes = []

for customer in customerp['CustomerID'].head(20):
    rec = recommendc(customer, customerp)
    rec['CustomerID'] = customer
    toplookalikes.append(rec)

lookalikesdf = pd.concat(toplookalikes, ignore_index=True)
lookalikesdf.to_csv('toplookalikes.csv', index=False)